In [21]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

In [22]:
bank_add_full = pd.read_csv('./bank-additional/bank-additional-full.csv',sep = ';')

In [23]:
# bank_full.columns
# 查看缺失值的情况
def viewNone(frame):
    for column in frame.columns:
#         print(type(bank_full[column][0]))
        if type(frame[column][0]) is str:
            none_count = frame[frame[column] == 'unknown'][column].count()
            if(none_count > 0):
                print('columns {0} unknown count {1}'.format(column,none_count))

In [24]:
print('-------------------------------------')
print('None value condition in bank_add_full.csv')
viewNone(bank_add_full)
print('-------------------------------------')

-------------------------------------
None value condition in bank_add_full.csv
columns job unknown count 330
columns marital unknown count 80
columns education unknown count 1731
columns default unknown count 8597
columns housing unknown count 990
columns loan unknown count 990
-------------------------------------


In [25]:
bank_add_full.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [26]:
all_columns = bank_add_full.columns
all_columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [27]:
numeric_attrs = ['age', 'duration', 'campaign', 'pdays', 'previous','emp.var.rate', 'cons.price.idx', 'cons.conf.idx','euribor3m', 'nr.employed']
cate_attrs = ['default', 'housing', 'loan','poutcome', 'education', 'job', 'marital','contact', 'month','day_of_week']
              

In [28]:
# 类别数量小于500的删掉
cate_attrs_tmp = ['y'] # 用来做预测的属性值
for cate in cate_attrs:
    if(bank_add_full[bank_add_full[cate] == 'unknown'][cate].count() < 500): # 删掉
            bank_add_full = bank_add_full[bank_add_full[cate] != 'unknown']
    else:
        cate_attrs_tmp.append(cate) # 不确定的值太多，则进行预测

In [29]:
print(cate_attrs_tmp)


['y', 'default', 'housing', 'loan', 'education']


In [30]:
# 受教育程度越高数值越大
edu_values = ["illiterate", "basic.4y", "basic.6y", "basic.9y", "high.school",  "professional.course", "university.degree"]
edu_dict = {edu:i for i,edu in enumerate(edu_values)}
for edu_value_i in edu_values:
    bank_add_full.loc[bank_add_full['education'] == edu_value_i,'education'] = edu_dict[edu_value_i]
# 将default ,housing ,loan 中的yes转为1 no 转为 0
for cate_attrs_tmp_i in cate_attrs_tmp[:-1]:
    bank_add_full.loc[bank_add_full[cate_attrs_tmp_i] == 'yes',cate_attrs_tmp_i] = 1
    bank_add_full.loc[bank_add_full[cate_attrs_tmp_i] == 'no',cate_attrs_tmp_i] = 0


In [31]:
# one hot encoding
one_hot_attrs = list(set(cate_attrs).difference(cate_attrs_tmp))
print(one_hot_attrs)
for one_hot_attr in one_hot_attrs:
    col_tmp = pd.get_dummies(bank_add_full[one_hot_attr])
    bank_add_full = bank_add_full.drop(one_hot_attr, axis = 1)
    bank_add_full = pd.concat([bank_add_full, col_tmp], axis = 1)

['contact', 'poutcome', 'job', 'marital', 'day_of_week', 'month']


In [32]:
# 对'age'属性进行分bin
bining_num = 10
bining_attr = 'age'
bank_add_full[bining_attr] = pd.qcut(bank_add_full[bining_attr], bining_num)
bank_add_full[bining_attr] = pd.factorize(bank_add_full[bining_attr])[0] + 1
stand = preprocessing.StandardScaler()

In [33]:
bank_add_full['age'].value_counts()

6     5087
2     4634
8     4400
4     4251
5     4188
3     3855
9     3735
10    3656
1     3513
7     3468
Name: age, dtype: int64

In [34]:
# 对数值属性进行归一化处理
bank_add_full[numeric_attrs] = stand.fit_transform(bank_add_full[numeric_attrs]) # must be list

In [35]:
# 对未知属性值做个预测
cate_attrs_tmp.remove('y')
for unknown_cate in cate_attrs_tmp:  
    train_data = bank_add_full[bank_add_full[unknown_cate] != 'unknown']
    train_y = train_data[unknown_cate]
    
    train_x = train_data.drop(cate_attrs_tmp, axis = 1)
    test_data = bank_add_full[bank_add_full[unknown_cate] == 'unknown']
    test_x = test_data.drop(cate_attrs_tmp, axis = 1) 
    
    train_y = train_y.astype('int')
    predict_y = RandomForestClassifier(n_estimators=200).fit(train_x,train_y).predict(test_x).astype(int)
    
    test_data.loc[:,unknown_cate] = predict_y
    bank_add_full = pd.concat([train_data,test_data]) # axis = 0
    

/Users/icdi/Desktop/py_ws/venv/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/icdi/Desktop/py_ws/venv/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/icdi/Desktop/py_ws/venv/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [36]:
for one_hot_attr in cate_attrs_tmp:
    col_tmp = pd.get_dummies(bank_add_full[one_hot_attr])
    columns = [ one_hot_attr + str(col) for col in (col_tmp.columns)]
    col_tmp.columns = columns
    bank_add_full = bank_add_full.drop(one_hot_attr, axis = 1)
    bank_add_full = pd.concat([bank_add_full, col_tmp], axis = 1)

In [37]:
bank_add_full.to_csv('preprocess_result.csv',index = False) # 预处理的结果